In [ ]:
import ExKMC.Tree as exkmc

In [ ]:
from graph_utils import get_nearest_neighbors
from init_tree import init_tree_by_name
from load_datasets import load_by_name

import numpy as np
import pandas as pd
from sklearn.cluster import SpectralClustering
from sklearn.metrics import adjusted_mutual_info_score, adjusted_rand_score, pairwise_distances, pairwise_distances_argmin_min, confusion_matrix
from sklearn.preprocessing import StandardScaler

In [ ]:
dataset_name = '20newsgroups' # See the load_by_name function for the available datasets
path = None # Necessary for some datasets

In [ ]:
data, clustering_true = load_by_name(dataset_name, path)
k = np.unique(clustering_true).size
k_prime = 2 * k

# Helper functions

In [ ]:
def kmeans_assignment(data, labels, centers):
    assert data.shape[0] == labels.size
    assert data.shape[1] == centers.shape[1]
    map_label_center = {}
    cost = 0
    distances = pairwise_distances(data, centers, metric='sqeuclidean')
    assert distances.shape[0] == data.shape[0]
    assert distances.shape[1] == centers.shape[0]

    unique_labels = np.unique(labels)
    for label in unique_labels:
        distances_for_cluster = distances[labels == label, :]
        distances_per_center = distances_for_cluster.sum(axis=0)
        map_label_center[label] = distances_per_center.argmin()
        cost += distances_per_center.min()

    return np.array([map_label_center[label] for label in labels]), cost

In [ ]:
# Maps each predicted cluster to the true class with which it shares
# the most members. This is needed to have a somewhat fair comparison between
# methods with a kMeans reference, for which an easy "matching is available a-la ExKMC, and
# those with a spectral reference, for which no such matching is available.
def label_matching(y_true, y_pred):
    true_labels = np.unique(y_true)
    pred_labels = np.unique(y_pred)

    cm = confusion_matrix(y_true, y_pred, labels=pred_labels)

    best_true_indices = np.argmax(cm, axis=0)
    best_true_labels = true_labels[best_true_indices]

    mapping = {pred: true for pred, true in zip(pred_labels, best_true_labels)}

    y_pred_mapped = np.array([mapping[pred] for pred in y_pred])

    mapped_labels = np.unique(y_pred_mapped)
    if len(mapped_labels) != len(true_labels):
        print("WARNING: Some clusters were not assigned any points.")

    return y_pred_mapped

# Train

In [ ]:
results = {}

clique_baselines = {}

def add_result(name, prediction):
    results[name] = [adjusted_rand_score(clustering_true, prediction), adjusted_mutual_info_score(clustering_true, prediction)]

## ExKMC (and KMeans)

In [ ]:
tree = exkmc.Tree(k=k, max_leaves=k_prime)

exkmc_prediction = tree.fit_predict(data)
centers = tree.all_centers

In [ ]:
clustering_kmeans, ref_cost = pairwise_distances_argmin_min(data, centers, metric='sqeuclidean')
add_result('K Means', clustering_kmeans)
clique_baselines['KMeans'] = clustering_kmeans

In [ ]:
add_result('ExKMC', exkmc_prediction)

## Spectral Clustering

In [ ]:
n_ngbrs = 50

scaler = StandardScaler()
scaler.fit(data)

scaled_data = scaler.transform(data)

In [ ]:
spectral = SpectralClustering(n_clusters=k, n_neighbors=n_ngbrs,
                                affinity='nearest_neighbors', assign_labels='cluster_qr',
                                random_state=570, eigen_solver='amg')
clustering_spectral = spectral.fit_predict(scaled_data)

In [ ]:
graph_nn = get_nearest_neighbors(scaled_data, n_ngbrs)

In [ ]:
clique_baselines['Spectral'] = clustering_spectral
add_result('Spectral Clustering', clustering_spectral)

In [ ]:
graph_global = init_tree_by_name('graph')
graph_global.train(data, graph_nn, k_prime)
add_result(f'SpEx kNN', label_matching(clustering_spectral, graph_global.predict(data)))

## SpEx Clique

In [ ]:
for name, reference in clique_baselines.items():
    clique_global = init_tree_by_name('clique')
    clique_global.train(data, reference, k=k_prime)
    prediction = clique_global.predict(data)
    if name == 'KMeans':
        prediction_matched = label_matching(reference, prediction)
        add_result(f'Clique Global {name} matched', prediction_matched)
        prediction = kmeans_assignment(data, prediction, centers)[0]
    if name == 'Spectral':
        prediction = label_matching(reference, prediction)
    add_result(f'SpEx Clique {name}', prediction)

## CART

In [ ]:
for name, reference in clique_baselines.items():
    vanilla_cart = init_tree_by_name('cart')
    vanilla_cart.train(data, reference, k=k_prime)
    prediction = vanilla_cart.predict(data)
    if name == 'KMeans':
        prediction_matched = label_matching(reference, prediction)
        add_result(f'CART {name} matched', prediction_matched, reference)
        prediction = kmeans_assignment(data, prediction, centers)[0]
    if name == 'Spectral':
        prediction = label_matching(clustering_spectral, prediction)
    add_result(f'CART {name}', prediction, reference)

# Evaluation

In [ ]:
metric_names = ["Rand Score", "AMI", "Rand Score w.r.t Baseline"]

In [ ]:
valuation_df = pd.DataFrame.from_dict(results, orient='index', columns=metric_names)
valuation_df